# Data Cleaning

In [1]:
#Import dependenices
import csv
import pandas as pd
from pathlib import Path
import hvplot.pandas

In [4]:
# Specify the path to your CSV file
csv_file_path1 = '../Resources/Credit_train.csv'
csv_file_path2 = '../Resources/Credit_test.csv'

# Read the CSV file into a DataFrame
loan_data_train = pd.read_csv(csv_file_path1)
loan_data_test = pd.read_csv(csv_file_path2)

#Print size of data set
print(loan_data_train.shape)
print(loan_data_test.shape)

# Display the DataFrame
loan_data_test.head(5)

(100514, 19)
(10353, 18)


,Loan ID,Customer ID,Current Loan Amount,Term,Credit Score,Annual Income,Years in current job,Home Ownership,Purpose,Monthly Debt,Years of Credit History,Months since last delinquent,Number of Open Accounts,Number of Credit Problems,Current Credit Balance,Maximum Open Credit,Bankruptcies,Tax Liens
0,f738779f-c726-40dc-92cf-689d73af533d,ded0b3c3-6bf4-4091-8726-47039f2c1b90,611314.0,Short Term,747.0,2074116.0,10+ years,Home Mortgage,Debt Consolidation,42000.83,21.8,NaN,9.0,0.0,621908.0,1058970.0,0.0,0.0
1,6dcc0947-164d-476c-a1de-3ae7283dde0a,1630e6e3-34e3-461a-8fda-09297d3140c8,266662.0,Short Term,734.0,1919190.0,10+ years,Home Mortgage,Debt Consolidation,36624.40,19.4,NaN,11.0,0.0,679573.0,904442.0,0.0,0.0
2,f7744d01-894b-49c3-8777-fc6431a2cff1,2c60938b-ad2b-4702-804d-eeca43949c52,153494.0,Short Term,709.0,871112.0,2 years,Rent,Debt Consolidation,8391.73,12.5,10.0,10.0,0.0,38532.0,388036.0,0.0,0.0
3,83721ffb-b99a-4a0f-aea5-ef472a138b41,12116614-2f3c-4d16-ad34-d92883718806,176242.0,Short Term,727.0,780083.0,10+ years,Rent,Debt Consolidation,16771.87,16.5,27.0,16.0,1.0,156940.0,531322.0,1.0,0.0
4,08f3789f-5714-4b10-929d-e1527ab5e5a3,39888105-fd5f-4023-860a-30a3e6f5ccb7,321992.0,Short Term,744.0,1761148.0,10+ years,Home Mortgage,Debt Consolidation,39478.77,26.0,44.0,14.0,0.0,359765.0,468072.0,0.0,0.0


In [6]:
clean_test = loan_data_test.applymap(lambda x: x.lower() if isinstance(x, str) else x)
clean_train = loan_data_train.applymap(lambda x: x.lower() if isinstance(x, str) else x)

In [8]:
#Drop columns which are not required for analysis
clean_train = clean_train.drop(columns=['Loan ID','Customer ID','Years in current job', 'Loan Status', 'Months since last delinquent', 'Tax Liens', 'Purpose'])
clean_test = clean_test.drop(columns=['Loan ID','Customer ID','Years in current job', 'Months since last delinquent', 'Tax Liens', 'Purpose'])

In [9]:
#Drop all null values from data frame
clean_test.dropna(inplace=True)
clean_train.dropna(inplace=True)

#Print size of data set
print(clean_train.shape)

(80683, 12)


In [10]:
# Verify the categories of the 'Term','Home Onwership' and 'Purpose'column
print('--------')
print(clean_train['Term'].value_counts())
print('--------')
print(clean_train['Home Ownership'].value_counts())

--------
short term    57577
long term     23106
Name: Term, dtype: int64
--------
home mortgage    39033
rent             34001
own home          7464
havemortgage       185
Name: Home Ownership, dtype: int64


In [12]:
# Verify the categories of the 'Term','Home Onwership' and 'Purpose'column
print('--------')
print(clean_test['Term'].value_counts())
print('--------')
print(clean_test['Home Ownership'].value_counts())

--------
short term    5785
long term     2215
Name: Term, dtype: int64
--------
home mortgage    3869
rent             3395
own home          736
Name: Home Ownership, dtype: int64


In [13]:
clean_train = clean_train.replace({'havemortgage': 'home mortgage'})
print(clean_train['Home Ownership'].value_counts())


home mortgage    39218
rent             34001
own home          7464
Name: Home Ownership, dtype: int64


In [15]:
# Verify the categories of the 'Term' column
print(clean_test['Term'].value_counts())
print(clean_train['Term'].value_counts())

short term    5785
long term     2215
Name: Term, dtype: int64
short term    57577
long term     23106
Name: Term, dtype: int64


In [16]:
# Transform the Home Ownership column using get_dummies
home_dummies1 = pd.get_dummies(clean_test['Home Ownership'])
home_dummies2 = pd.get_dummies(clean_train['Home Ownership'])
# Display the transformed data
home_dummies2.tail()

,home mortgage,own home,rent
99995,0,1,0
99996,0,0,1
99997,0,0,1
99998,0,0,1
99999,0,1,0


In [17]:
# Transform the Term column using get_dummies
term_dummies1 = pd.get_dummies(clean_test['Term'])
term_dummies2 = pd.get_dummies(clean_train['Term'])

# Display the transformed data
term_dummies1.tail()

,long term,short term
9994,0,1
9995,0,1
9996,0,1
9998,0,1
9999,0,1


In [18]:
# Concatenate the df_shopping_transformed and the card_dummies DataFrames
clean_test = pd.concat([clean_test, term_dummies1], axis = 1)
clean_test = pd.concat([clean_test, home_dummies1], axis = 1)
clean_train = pd.concat([clean_train, term_dummies2], axis = 1)
clean_train = pd.concat([clean_train, home_dummies2], axis = 1)

# Drop the original column
clean_test = clean_test.drop(columns=['Term'])
clean_test = clean_test.drop(columns=['Home Ownership'])
clean_train = clean_train.drop(columns=['Term'])
clean_train = clean_train.drop(columns=['Home Ownership'])
# Display the DataFrame
clean_test.head()

,Current Loan Amount,Credit Score,Annual Income,Monthly Debt,Years of Credit History,Number of Open Accounts,Number of Credit Problems,Current Credit Balance,Maximum Open Credit,Bankruptcies,long term,short term,home mortgage,own home,rent
0,611314.0,747.0,2074116.0,42000.83,21.8,9.0,0.0,621908.0,1058970.0,0.0,0,1,1,0,0
1,266662.0,734.0,1919190.0,36624.40,19.4,11.0,0.0,679573.0,904442.0,0.0,0,1,1,0,0
2,153494.0,709.0,871112.0,8391.73,12.5,10.0,0.0,38532.0,388036.0,0.0,0,1,0,0,1
3,176242.0,727.0,780083.0,16771.87,16.5,16.0,1.0,156940.0,531322.0,1.0,0,1,0,0,1
4,321992.0,744.0,1761148.0,39478.77,26.0,14.0,0.0,359765.0,468072.0,0.0,0,1,1,0,0


In [19]:
# Capitalise the headings
clean_test = clean_test.rename(columns=lambda x: x.capitalize())
clean_train = clean_train.rename(columns=lambda x: x.capitalize())


In [20]:
# Change Bankruptcies column into a boolean value
# Define the condition
condition1 = clean_test['Bankruptcies'] > 0
condition2 = clean_train['Bankruptcies'] > 0

# Create a new boolean column based on the condition
clean_test['Bankrupt'] = condition1
clean_train['Bankrupt'] = condition2

# Convert to numerical values
clean_test['Bankrupt'] = clean_test['Bankrupt'].astype(int)
clean_train['Bankrupt'] = clean_train['Bankrupt'].astype(int)

#Drop Bankruptcies column
clean_test = clean_test.drop('Bankruptcies', axis = 1)
clean_train = clean_train.drop('Bankruptcies', axis = 1)

In [21]:
# Display the test DataFrame
clean_test.head()

,Current loan amount,Credit score,Annual income,Monthly debt,Years of credit history,Number of open accounts,Number of credit problems,Current credit balance,Maximum open credit,Long term,Short term,Home mortgage,Own home,Rent,Bankrupt
0,611314.0,747.0,2074116.0,42000.83,21.8,9.0,0.0,621908.0,1058970.0,0,1,1,0,0,0
1,266662.0,734.0,1919190.0,36624.40,19.4,11.0,0.0,679573.0,904442.0,0,1,1,0,0,0
2,153494.0,709.0,871112.0,8391.73,12.5,10.0,0.0,38532.0,388036.0,0,1,0,0,1,0
3,176242.0,727.0,780083.0,16771.87,16.5,16.0,1.0,156940.0,531322.0,0,1,0,0,1,1
4,321992.0,744.0,1761148.0,39478.77,26.0,14.0,0.0,359765.0,468072.0,0,1,1,0,0,0


In [22]:
# Display the test DataFrame
clean_train.head()

,Current loan amount,Credit score,Annual income,Monthly debt,Years of credit history,Number of open accounts,Number of credit problems,Current credit balance,Maximum open credit,Long term,Short term,Home mortgage,Own home,Rent,Bankrupt
0,445412.0,709.0,1167493.0,5214.74,17.2,6.0,1.0,228190.0,416746.0,0,1,1,0,0,1
2,99999999.0,741.0,2231892.0,29200.53,14.9,18.0,1.0,297996.0,750090.0,0,1,0,1,0,0
3,347666.0,721.0,806949.0,8741.90,12.0,9.0,0.0,256329.0,386958.0,1,0,0,1,0,0
5,206602.0,7290.0,896857.0,16367.74,17.3,6.0,0.0,215308.0,272448.0,0,1,1,0,0,0
6,217646.0,730.0,1184194.0,10855.08,19.6,13.0,1.0,122170.0,272052.0,0,1,1,0,0,1


# Data Scaling

In [23]:
from sklearn.preprocessing import StandardScaler

In [24]:
# Scaling the numeric columns
train_scaled = StandardScaler().fit_transform(clean_train[['Current loan amount', 'Credit score', 'Annual income','Monthly debt','Years of credit history','Number of open accounts', 'Number of credit problems','Current credit balance','Maximum open credit']])

# Review the train scaled data
train_scaled

array([[-4.03050203e-01, -2.48996183e-01, -1.95064699e-01, ...,
         1.69937754e+00, -1.75311853e-01, -4.60753246e-02],
       [ 2.45806717e+00, -2.27301607e-01,  7.89666434e-01, ...,
         1.69937754e+00,  7.62938426e-03,  2.27017159e-03],
       [-4.05859343e-01, -2.40860717e-01, -5.28622793e-01, ...,
        -3.48918343e-01, -1.01567712e-01, -5.03955333e-02],
       ...,
       [-4.12886935e-01, -2.26623651e-01, -2.10744180e-01, ...,
         1.69937754e+00, -4.86222327e-01, -2.85551874e-02],
       [-4.00609671e-01, -2.23911829e-01,  3.13797720e-01, ...,
        -3.48918343e-01,  2.86024718e-01,  5.53574639e-04],
       [ 2.45806717e+00, -2.25945696e-01, -4.09989595e-01, ...,
         1.69937754e+00, -6.53827173e-01, -9.33168384e-02]])

In [25]:
# Scaling the numeric columns
test_scaled = StandardScaler().fit_transform(clean_test[['Current loan amount', 'Credit score', 'Annual income','Monthly debt','Years of credit history','Number of open accounts', 'Number of credit problems','Current credit balance','Maximum open credit']])

# Review the train scaled data
test_scaled

array([[-0.39670955, -0.22436593,  0.81018507, ..., -0.33242601,
         0.86821277,  0.40269415],
       [-0.40663987, -0.23315551,  0.63193605, ..., -0.33242601,
         1.01942387,  0.24773424],
       [-0.40990053, -0.25005854, -0.57392278, ..., -0.33242601,
        -0.66153538, -0.27011512],
       ...,
       [-0.41050398, -0.24397345, -0.69808938, ..., -0.33242601,
        -0.56771973, -0.3286222 ],
       [ 2.46693464, -0.24194509, -0.45773519, ...,  1.67619036,
        -0.27750403, -0.41790505],
       [ 2.46693464, -0.22368981, -0.33363417, ..., -0.33242601,
        -0.29314827, -0.04965465]])

In [20]:
# Create a DataFrame of the scaled data
train_scaled = pd.DataFrame(train_scaled, columns=['Current loan amount', 'Credit score', 'Annual income','Monthly debt','Years of credit history','Number of open accounts', 'Number of credit problems','Current credit balance','Maximum open credit'])

# Replace the original data with the columns of information from the scaled Data
clean_train['Current loan amount'] = train_scaled['Current loan amount']
clean_train['Credit score'] = train_scaled['Credit score']
clean_train['Annual income'] = train_scaled['Annual income']
clean_train['Monthly debt'] = train_scaled['Monthly debt']
clean_train['Years of credit history'] = train_scaled['Years of credit history']
clean_train['Number of open accounts'] = train_scaled['Number of open accounts']
clean_train['Number of credit problems'] = train_scaled['Number of credit problems']
clean_train['Current credit balance'] = train_scaled['Current credit balance']
clean_train['Maximum open credit'] = train_scaled['Maximum open credit']

# Review the DataFrame
clean_train.head()

,Current loan amount,Credit score,Annual income,Monthly debt,Years of credit history,Number of open accounts,Number of credit problems,Current credit balance,Maximum open credit,Long term,Short term,Home mortgage,Own home,Rent,Bankrupt
0,-0.396710,-0.224366,0.810185,1.930978,0.519773,-0.416087,-0.332426,0.868213,0.402694,0,1,1,0,0,0
1,-0.406640,-0.233156,0.631936,1.489873,0.174878,-0.016795,-0.332426,1.019424,0.247734,0,1,1,0,0,0
2,-0.409901,-0.250059,-0.573923,-0.826455,-0.816695,-0.216441,-0.332426,-0.661535,-0.270115,0,1,0,0,1,0
3,-0.409245,-0.237888,-0.678656,-0.138913,-0.241870,0.981434,1.676190,-0.351042,-0.126429,0,1,0,0,1,1
4,-0.405046,-0.226394,0.450102,1.724058,1.123338,0.582142,-0.332426,0.180813,-0.189855,0,1,1,0,0,0


In [27]:
# Create a DataFrame of the scaled data
test_scaled = pd.DataFrame(test_scaled, columns=['Current loan amount', 'Credit score', 'Annual income','Monthly debt','Years of credit history','Number of open accounts', 'Number of credit problems','Current credit balance','Maximum open credit'])

# Replace the original data with the columns of information from the scaled Data
clean_test['Current loan amount'] = test_scaled['Current loan amount']
clean_test['Credit score'] = test_scaled['Credit score']
clean_test['Annual income'] = test_scaled['Annual income']
clean_test['Monthly debt'] = test_scaled['Monthly debt']
clean_test['Years of credit history'] = test_scaled['Years of credit history']
clean_test['Number of open accounts'] = test_scaled['Number of open accounts']
clean_test['Number of credit problems'] = test_scaled['Number of credit problems']
clean_test['Current credit balance'] = test_scaled['Current credit balance']
clean_test['Maximum open credit'] = test_scaled['Maximum open credit']

# Review the DataFrame
clean_test.head()

,Current loan amount,Credit score,Annual income,Monthly debt,Years of credit history,Number of open accounts,Number of credit problems,Current credit balance,Maximum open credit,Long term,Short term,Home mortgage,Own home,Rent,Bankrupt
0,-0.396710,-0.224366,0.810185,1.930978,0.519773,-0.416087,-0.332426,0.868213,0.402694,0,1,1,0,0,0
1,-0.406640,-0.233156,0.631936,1.489873,0.174878,-0.016795,-0.332426,1.019424,0.247734,0,1,1,0,0,0
2,-0.409901,-0.250059,-0.573923,-0.826455,-0.816695,-0.216441,-0.332426,-0.661535,-0.270115,0,1,0,0,1,0
3,-0.409245,-0.237888,-0.678656,-0.138913,-0.241870,0.981434,1.676190,-0.351042,-0.126429,0,1,0,0,1,1
4,-0.405046,-0.226394,0.450102,1.724058,1.123338,0.582142,-0.332426,0.180813,-0.189855,0,1,1,0,0,0


In [28]:
# Export scaled data
csv_file_path = '../Resources/Clean_train_scaled.csv'

clean_train.to_csv(csv_file_path, index=False)

# Export scaled data
csv_file_path = '../Resources/Clean_test_scaled.csv'

clean_test.to_csv(csv_file_path, index=False)